# Introduction


- In this project, we are investigating how different AI agents behave in a simple sealed-bid auction, which is where each agent gets a private value and has to decide how much to bid without seeing anyone else's bids. The main question here is *Given different goals and personalities, how do these agents choose their bids and what kinds of outcomes does that produce?*

- This is interesting from a computational social science perspective because auctions are everywhere (ads, markets, resource allocation, etc) and they are a clean way to study strategic behavior.

- In real auctions, people use shortcuts, worry about risk, try to outsmart each other, or just behave unpredictably. This project mixes AI-generated reasoning with actual computational tools like payoff calculators and simulations. The idea is to see what happens when you combine classical game-theoretic structure with LLM-style decision making. It ends up creating a small ecosystem where different agent types have to reason, compete, and adapt, giving us a more realistic picture than purely analytical models.



# System Design

The system is built around three main agent types running repeated sealed-bid auctions: an AuctioneerAgent that manages the environment, and two bidder types—StrategicBidderAgent and HeuristicBidderAgent—that take different approaches to bidding.

## The Agents

1. **The Auctioneer** is the coordinator.

    - Samples private values for each bidder from a configured distribution.

    - Collects bids, determines the winner and clearing price, and calls tools to compute utilities.

    - Sends back a compact outcome summary each round, which is also used later for evaluation and plots.
    
2. **The Strategic Bidder** is meant to represent a more “game-theoretic” participant:

    - It is roughly risk-neutral and tries to maximize expected utility.

    - It receives its private value and a simple belief model about how others bid.

    - It uses a best-response tool to scan through candidate bids, simulate many hypothetical auctions, and pick a bid that approximately maximizes expected utility.

3. **The Heuristic Bidder** is deliberately less sophisticated:

    - It follows simple rules like “shade my bid by a fixed factor” or “slightly overbid my value.”

    - It still has access to payoff calculations but does not run full search/optimization.

    - Over multiple auctions, it can update its shading parameter based on how often it wins and how much utility it gets, mimicking a rule-of-thumb learner instead of a full optimizer.

## The Tools
1. Payoff Calculator

    - The AuctioneerAgent uses this to finalize the outcome of a real auction round, and bidders can call it to ask: “If I bid X and others behave like this, what would my payoff be?”

2. Best-Response Approximator

    - The StrategicBidderAgent relies on this to turn “game-theoretic intent” into an actual numeric decision.

3. Auction Simulator

    - This simulator is mainly used by the Auctioneer. It runs many auctions in a row under fixed strategy profiles to get empirical stats (instead of just one-off outcomes).

## Interaction Protocol
The interaction pattern is a simple single-round loop that can be repeated for many auctions:

1. Auction Setup

    - The Auctioneer samples private values for all bidders from a specified distribution (e.g., uniform on [0, 1]).

    - It builds an AuctionConfig object (auction type, number of bidders, value distribution) and sends each bidder a BidRequest containing:

        - Their private value

        - The auction configuration

        - Optional history of past outcomes (for learning/adjustment)

2. Bid Generation

    - Each bidder processes the BidRequest with its own logic:

        - The StrategicBidderAgent calls the best-response tool to search over candidate bids.

        - The HeuristicBidderAgent applies its rule-of-thumb strategy, possibly updating its shading factor using past outcomes.

    - Each agent returns a BidResponse with: bidder_id, bid_amount,  A short reasoning_summary (for logging/interpretability).

    - Outcome Computation

        - The Auctioneer collects all BidResponses, decides the winner and price based on the auction type, and calls the payoff calculator.

        - The tool returns an AuctionOutcome with: winner_id, clearing_price, bids, values, payments, and per-agent payoff.

4. Feedback and Logging

    - The Auctioneer broadcasts a summary (who won, price, each bidder’s payoff) that can be stored for evaluation and optionally fed back into future BidRequest.history for learning.

Running this loop many times creates a dataset of auctions that you can analyze: distributions of bids, payoffs, win rates by agent type, and how behavior changes over time.

## Design Choices and Tradeoffs Discussion


### 1) LLM-Based Decision Making

All agent types utilize LLM reasoning.

Advantages:
* We gain the ability to understand the reasoning for each agents bid, rather than a hard-coded outcome.
* Ideally, each agent has the ability to learn from previous rounds and be more competitive.

Disadvantages:
* Much more expensive to have three LLM calls per round, for 20 rounds
* Runtime is much longer (not too overbearing for 20 rounds, but probably would not scale well)
* Lack of reproducibility
* LLM hallucination and reasoning failures can cause agents to behave unexpectedly

### 2) Allowing HeuristicAgent to dynamically change shading factor (0.7 - 0.95)

Even though the HeuristicAgent should follow a "rule-of-thumb" strategy, our simulation allows small adjustments based on history.

Advantages:
* Can see how reactive the agent is to positive or negative outcomes, if at all.
* Adds complexity and realism, preventing agents from being stuck.
  
Disadvantages:
* The LLM's interpretation of "when to adjust" may be unclear, even with reasoning provided.
* Over only 20 rounds, the agents tended to not show drastic adjustments.
* This functionality is the reason HeuristicAgent requires LLM interaction.

### 3) 3 Total Bidders, 1 strategic and 2 heuristic

Advantages:
* Simpler
* Cheaper
* Faster

Disadvantages:
* May lose interesting insights and bidding trends when more factors are involved

### 4) Only Test Sealed-Bid Second-Price (Vickrey) Auction

We do not test other auction types (first-price, English, etc)

Advantages:
* Focus on one auction type for implementation and interpretation
* Simple payoff structure and auction setup

Disadvantages:
* May lose insights into how the different types of agents respond to different setups

### 5) What is included in "History"? 

The history that is passed to each agent each round contains:
* Previous Bids from all agents
* Previous Private Values from all agents
* Winner ID
* Round Index

Details not *explicitly* included (although maybe implicit)
* Payoffs
* Clearing Price
* Shading factors

Relying on the LLM to correctly infer all the omitted information could result in incorrect choices/reasonings.

# Implementation

Show key code components

Demonstrate tool definitions

Include example interactions

Show state management

# Experiments + Results

Present multiple scenarios or experiments

Include visualizations where appropriate

Show agent reasoning (chain-of-thought outputs)

Report evaluation results

# Analysis + Discussion

Interpret your results

Connect to course concepts (game theory, emergence, networks, equilibria)

Discuss limitations and future extensions

Reflect on what you learned

# Conclusion

Summarize key findings

Implications for computational social science